In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import TomekLinks
from collections import Counter
from sklearn.preprocessing import StandardScaler # Exemplo de pré-processamento
from sklearn.neighbors import KNeighborsClassifier # Exemplo de modelo

# --- VARIÁVEIS DE CONFIGURAÇÃO DO PROJETO ARTRITE ---
# 🚨 Substitua pelo nome correto da sua base imputada
NOME_ARQUIVO_ENTRADA = "BaseFinal(v1).csv" 
# 🚨 Substitua pelo nome correto da sua coluna alvo/dependente (Y)
NOME_COLUNA_TARGET = 'TEM_ARTRITE' 
# ---------------------------------------------------

try:
    # 1. Carregamento da Base
    print(f"--- 1. Carregando a Base de Dados: {NOME_ARQUIVO_ENTRADA} ---")
    # Nota: Ajuste o 'sep' se sua base não for lida corretamente.
    base = pd.read_csv(NOME_ARQUIVO_ENTRADA) 
    
    # 2. Definição de Features (X) e Target (y)
    if NOME_COLUNA_TARGET not in base.columns:
        NOME_COLUNA_TARGET = base.columns[-1]
        print(f"Alerta: Coluna alvo não encontrada, usando a última coluna: '{NOME_COLUNA_TARGET}'.")

    X_prev = base.drop(columns=[NOME_COLUNA_TARGET])
    y_classe = base[NOME_COLUNA_TARGET]
    
    print(f"Classes Originais: {Counter(y_classe)}")
    print("-" * 50)
    
    # 3. Dividir em conjunto de treino e teste (A prática é não balancear o teste)
    X_treino, X_teste, y_treino, y_teste = train_test_split(
        X_prev, 
        y_classe, 
        test_size=0.20,
        random_state=42,
        # Estratificação garante que as proporções de classe sejam mantidas em treino/teste
        stratify=y_classe 
    )
    print(f"Classes Treino (Original): {Counter(y_treino)}")
    print(f"Classes Teste (Desbalanceado): {Counter(y_teste)}")
    print("-" * 50)
    
    # 4. Etapas de Pré-Processamento (Ajuste/Inclua o que for necessário aqui!)
    print("--- 4. Aplicando Pré-Processamento (Ex: Padronização) ---")
    # Exemplo: Padronização/Normalização das Features é CRUCIAL para KNN e TomekLinks
    scaler = StandardScaler()
    
    # Fit (aprender) apenas no treino e depois Transformar treino e teste
    X_treino_scaled = scaler.fit_transform(X_treino)
    X_teste_scaled = scaler.transform(X_teste) 
    
    X_treino_df = pd.DataFrame(X_treino_scaled, columns=X_treino.columns)
    X_teste_df = pd.DataFrame(X_teste_scaled, columns=X_teste.columns)
    
    print("Padronização/Escalonamento concluído.")
    print("-" * 50)


    # 5. Balanceamento com TomekLinks (Apenas no conjunto de TREINO)
    print("--- 5. Aplicando Undersampling: TomekLinks ---")
    
    balanceamento_under = TomekLinks(sampling_strategy='auto')
    
    # Aplica o undersampling nas features escaladas do treino
    X_under, y_under = balanceamento_under.fit_resample(X_treino_df, y_treino)
    
    print(f"Classes Treino (APÓS TomekLinks): {Counter(y_under)}")
    print(f"Número de amostras removidas pela classe majoritária: {len(y_treino) - len(y_under)}")
    print("-" * 50)
    
    # 6. Treinar o Modelo (Exemplo com KNN)
    print("--- 6. Treinando o Modelo (Exemplo: KNN) ---")
    
    # Treina o modelo com os dados de treino balanceados (X_under, y_under)
    modelo_knn = KNeighborsClassifier(n_neighbors=5)
    modelo_knn.fit(X_under, y_under)
    
    print("Modelo KNN treinado com sucesso na base TomekLinks.")
    print("-" * 50)
    
    # 7. Avaliar o Modelo (usando o conjunto de TESTE desbalanceado)
    # A avaliação deve ser sempre no conjunto de TESTE original para métricas reais.
    from sklearn.metrics import classification_report
    
    y_pred = modelo_knn.predict(X_teste_df)
    
    print("Relatório de Classificação no Conjunto de Teste (Desbalanceado):")
    print(classification_report(y_teste, y_pred))
    
except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{NOME_ARQUIVO_ENTRADA}' não foi encontrado.")
except ImportError as e:
    print(f"❌ ERRO: Verifique se todas as bibliotecas estão instaladas. Erro: {e}")
except Exception as e:
    print(f"❌ Ocorreu um erro durante o processamento: {e}")

--- 1. Carregando a Base de Dados: BaseFinal(v1).csv ---
Classes Originais: Counter({1: 21542, 0: 1960})
--------------------------------------------------
Classes Treino (Original): Counter({1: 17233, 0: 1568})
Classes Teste (Desbalanceado): Counter({1: 4309, 0: 392})
--------------------------------------------------
--- 4. Aplicando Pré-Processamento (Ex: Padronização) ---
Padronização/Escalonamento concluído.
--------------------------------------------------
--- 5. Aplicando Undersampling: TomekLinks ---
Classes Treino (APÓS TomekLinks): Counter({1: 16737, 0: 1568})
Número de amostras removidas pela classe majoritária: 496
--------------------------------------------------
--- 6. Treinando o Modelo (Exemplo: KNN) ---
Modelo KNN treinado com sucesso na base TomekLinks.
--------------------------------------------------
Relatório de Classificação no Conjunto de Teste (Desbalanceado):
              precision    recall  f1-score   support

           0       0.25      0.08      0.12  

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.preprocessing import StandardScaler # Exemplo de pré-processamento
from sklearn.neighbors import KNeighborsClassifier # Exemplo de modelo
from sklearn.metrics import classification_report # Para avaliação

# --- VARIÁVEIS DE CONFIGURAÇÃO DO PROJETO ARTRITE ---
# 🚨 Substitua pelo nome correto da sua base imputada
NOME_ARQUIVO_ENTRADA = "BaseFinal(v1).csv" 
# 🚨 Substitua pelo nome correto da sua coluna alvo/dependente (Y)
NOME_COLUNA_TARGET = 'TEM_ARTRITE' 
# ---------------------------------------------------

try:
    # 1. Carregamento da Base
    print(f"--- 1. Carregando a Base de Dados: {NOME_ARQUIVO_ENTRADA} ---")
    base = pd.read_csv(NOME_ARQUIVO_ENTRADA) 
    
    # 2. Definição de Features (X) e Target (y)
    if NOME_COLUNA_TARGET not in base.columns:
        NOME_COLUNA_TARGET = base.columns[-1]
        print(f"Alerta: Coluna alvo não encontrada, usando a última coluna: '{NOME_COLUNA_TARGET}'.")

    X_prev = base.drop(columns=[NOME_COLUNA_TARGET])
    y_classe = base[NOME_COLUNA_TARGET]
    
    print(f"Classes Originais: {Counter(y_classe)}")
    print("-" * 50)
    
    # 3. Dividir em conjunto de treino e teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(
        X_prev, 
        y_classe, 
        test_size=0.20,
        random_state=42,
        stratify=y_classe 
    )
    print(f"Classes Treino (Original): {Counter(y_treino)}")
    print(f"Classes Teste (Desbalanceado): {Counter(y_teste)}")
    print("-" * 50)
    
    # 4. Etapas de Pré-Processamento (Padronização é crucial para KNN e RUS)
    print("--- 4. Aplicando Pré-Processamento (Ex: Padronização) ---")
    scaler = StandardScaler()
    
    # Fit apenas no treino
    X_treino_scaled = scaler.fit_transform(X_treino)
    X_teste_scaled = scaler.transform(X_teste) 
    
    X_treino_df = pd.DataFrame(X_treino_scaled, columns=X_treino.columns)
    X_teste_df = pd.DataFrame(X_teste_scaled, columns=X_teste.columns)
    
    print("Padronização/Escalonamento concluído.")
    print("-" * 50)


    # 5. Balanceamento com RandomUnderSampler (Apenas no conjunto de TREINO)
    print("--- 5. Aplicando Undersampling: RandomUnderSampler ---")
    
    # RandomUnderSampler: remove amostras aleatoriamente da classe majoritária
    undersample = RandomUnderSampler(random_state=42)
    
    X_resampled, y_resampled = undersample.fit_resample(X_treino_df, y_treino)
    
    print(f"Classes Treino (APÓS RandomUnderSampler): {Counter(y_resampled)}")
    print(f"Número de amostras removidas pela classe majoritária: {len(y_treino) - len(y_resampled)}")
    print("-" * 50)
    
    # 6. Treinar o Modelo (Exemplo com KNN)
    print("--- 6. Treinando o Modelo (Exemplo: KNN) ---")
    
    # Treina o modelo com os dados de treino balanceados (X_resampled, y_resampled)
    modelo_knn = KNeighborsClassifier(n_neighbors=5)
    modelo_knn.fit(X_resampled, y_resampled)
    
    print("Modelo KNN treinado com sucesso na base RandomUnderSampler.")
    print("-" * 50)
    
    # 7. Avaliar o Modelo (usando o conjunto de TESTE desbalanceado)
    y_pred = modelo_knn.predict(X_teste_df)
    
    print("Relatório de Classificação no Conjunto de Teste (Desbalanceado):")
    print(classification_report(y_teste, y_pred))
    
except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{NOME_ARQUIVO_ENTRADA}' não foi encontrado.")
except ImportError as e:
    print(f"❌ ERRO: Verifique se todas as bibliotecas estão instaladas. Erro: {e}")
except Exception as e:
    print(f"❌ Ocorreu um erro durante o processamento: {e}")

--- 1. Carregando a Base de Dados: BaseFinal(v1).csv ---
Classes Originais: Counter({1: 21542, 0: 1960})
--------------------------------------------------
Classes Treino (Original): Counter({1: 17233, 0: 1568})
Classes Teste (Desbalanceado): Counter({1: 4309, 0: 392})
--------------------------------------------------
--- 4. Aplicando Pré-Processamento (Ex: Padronização) ---
Padronização/Escalonamento concluído.
--------------------------------------------------
--- 5. Aplicando Undersampling: RandomUnderSampler ---
Classes Treino (APÓS RandomUnderSampler): Counter({0: 1568, 1: 1568})
Número de amostras removidas pela classe majoritária: 15665
--------------------------------------------------
--- 6. Treinando o Modelo (Exemplo: KNN) ---
Modelo KNN treinado com sucesso na base RandomUnderSampler.
--------------------------------------------------
Relatório de Classificação no Conjunto de Teste (Desbalanceado):
              precision    recall  f1-score   support

           0       0

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# --- VARIÁVEIS DE CONFIGURAÇÃO DO PROJETO ARTRITE ---
NOME_ARQUIVO_ENTRADA = "BaseFinal(v1).csv" 
NOME_COLUNA_TARGET = 'TEM_ARTRITE' 
# 🚨 NOVO: Nome do arquivo para salvar a base de TREINO balanceada
NOME_ARQUIVO_SMOTE_TREINO_SAIDA = "Base_SMOTE_Treino_Balanceada.csv"
# ---------------------------------------------------

try:
    # 1. Carregamento e Definição de Features/Target (Omissos para brevidade)
    base = pd.read_csv(NOME_ARQUIVO_ENTRADA) 
    X_prev = base.drop(columns=[NOME_COLUNA_TARGET])
    y_classe = base[NOME_COLUNA_TARGET]
    
    # 2. Divisão e Pré-Processamento (Omissos para brevidade)
    X_treino, X_teste, y_treino, y_teste = train_test_split(
        X_prev, y_classe, test_size=0.20, random_state=42, stratify=y_classe 
    )
    scaler = StandardScaler()
    X_treino_scaled = scaler.fit_transform(X_treino)
    X_teste_scaled = scaler.transform(X_teste) 
    X_treino_df = pd.DataFrame(X_treino_scaled, columns=X_treino.columns)
    X_teste_df = pd.DataFrame(X_teste_scaled, columns=X_teste.columns)


    # 5. Balanceamento com SMOTE
    print("--- 5. Aplicando Oversampling: SMOTE ---")
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_treino_df, y_treino)
    print(f"Classes Treino (APÓS SMOTE): {Counter(y_resampled)}")
    
    # 🚨 NOVO PASSO: SALVANDO A BASE DE TREINO BALANCEADA
    df_resampled = pd.concat([X_resampled, y_resampled], axis=1)
    df_resampled.to_csv(NOME_ARQUIVO_SMOTE_TREINO_SAIDA, index=False)
    print(f"✅ Base de TREINO balanceada (SMOTE) salva em: **{NOME_ARQUIVO_SMOTE_TREINO_SAIDA}**")
    print("-" * 50)
    
    # 6. Treinar e Avaliar o Modelo (As métricas continuam sendo medidas)
    print("--- 6. Treinando e Avaliando o Modelo (KNN com K=5) ---")
    
    modelo_knn = KNeighborsClassifier(n_neighbors=5)
    modelo_knn.fit(X_resampled, y_resampled)
    y_pred = modelo_knn.predict(X_teste_df)
    
    print("Relatório de Classificação no Conjunto de Teste (Desbalanceado):")
    print(classification_report(y_teste, y_pred))
    
except Exception as e:
    print(f"❌ Ocorreu um erro: {e}")

--- 5. Aplicando Oversampling: SMOTE ---
Classes Treino (APÓS SMOTE): Counter({1: 17233, 0: 17233})
✅ Base de TREINO balanceada (SMOTE) salva em: **Base_SMOTE_Treino_Balanceada.csv**
--------------------------------------------------
--- 6. Treinando e Avaliando o Modelo (KNN com K=5) ---
Relatório de Classificação no Conjunto de Teste (Desbalanceado):
              precision    recall  f1-score   support

           0       0.17      0.64      0.27       392
           1       0.96      0.71      0.81      4309

    accuracy                           0.70      4701
   macro avg       0.56      0.68      0.54      4701
weighted avg       0.89      0.70      0.77      4701

